In [ ]:
! pip install wandb

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project="mnist")

## **Dataset**

In [23]:
mnist = tf.keras.datasets.mnist

(X_train, Y_train), (X_rem, Y_rem) = mnist.load_data()
X_val, X_test, Y_val, Y_test = train_test_split(X_rem,Y_rem, test_size=0.5)

#normalization
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [24]:
model = tf.keras.models.Sequential([
    # استخراج ویژگی
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (5,5), activation='relu'),

    layers.Flatten(),

    #همون شبکه ساده قدیمی
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [25]:
config = wandb.config
config.learning_rate = 0.001

In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(config.learning_rate),
              loss = tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy']) 

In [27]:
model.fit(X_train, Y_train, epochs=5 , validation_data=(X_val, Y_val),callbacks=[WandbCallback()])

Epoch 1/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1482 - accuracy: 0.9550 - val_loss: 0.0460 - val_accuracy: 0.9854 - _timestamp: 1646838988.0000 - _runtime: 39.0000
Epoch 2/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0482 - accuracy: 0.9854 - val_loss: 0.0382 - val_accuracy: 0.9866 - _timestamp: 1646839001.0000 - _runtime: 52.0000
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0353 - accuracy: 0.9892 - val_loss: 0.0252 - val_accuracy: 0.9906 - _timestamp: 1646839013.0000 - _runtime: 64.0000
Epoch 4/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0271 - accuracy: 0.9912 - val_loss: 0.0418 - val_accuracy: 0.9862 - _timestamp: 1646839026.0000 - _runtime: 77.0000
Epoch 5/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0207 - accuracy: 0.9931 - val_loss: 0.0244 - val_accuracy: 0.9914 - _timestamp: 1646839038.0000 - _runtime: 89.0000


In [29]:
model.save("my_mnist.h5")

In [30]:
model.evaluate(X_test,Y_test)

157/157 [==============================] - 1s 4ms/step - loss: 0.0327 - accuracy: 0.9908


[0.03266192600131035, 0.9908000230789185]

In [31]:
img = cv2.imread("3.png")
img = cv2.resize(img, (28,28))

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img = img / 255.0
img = img.reshape(1,28,28)

result = model.predict([img])
np.argmax(result)

3